# Long-Range Fermi-Hubbard Model

The Hamiltonian is

\begin{equation}
H = - \sum_{ij\sigma} J_{ij}{\left(c^\dagger_{i,\sigma}c_{j,\sigma} + \rm{h.c.}\right)}  + \sum_{ij\sigma} U_{ij} \left(c^\dagger_{i\uparrow}c_{i\uparrow} + \frac{1}{2}\right)\left(c^\dagger_{j\downarrow}c_{j\downarrow} + \frac{1}{2}\right),
\end{equation}

where $U_{ij} = U\delta_{i,j}$ (on-site interaction) and $J_{ij} =  |i-j|^{-\beta}$ (long-range). We're starting from a state where the system is half-filled with the first half of the sites doubly occupied. The Truncated Wigner dynamics are given by

\begin{eqnarray}
\frac{d \rho^{\sigma\tau}}{d t}&=&-i\left[J, \rho^{\sigma\tau}\right]-U \rho^{\sigma\tau}\ast D\left(\rho^{-\sigma-\sigma},\rho^{-\tau-\tau}\right),\\
\frac{d \tau^{\sigma\tau}}{d t} &=& -i \big\{J\;\tau^{\sigma\tau}- \left(\tau^{\sigma\tau}\right)^T\;J\big\} + \big\{\tau^{\sigma\tau}\ast D\left(\rho^{-\sigma-\sigma},0\right)-\left(\tau^{\sigma\tau}\right)^T\ast D\left(\rho^{-\tau-\tau},0\right)\big\} + \frac{U}{2}\big\{\tau^{\sigma\tau}-\left(\tau^{\sigma\tau}\right)^T\big\}
\end{eqnarray}

where the asterix $\ast$ denotes element-by-element multiplication of matrices, and the diagonal difference matrix $D(A,B)_{ij} = A_{ii}-B_{jj}$. The indices $\sigma,\tau\in[\uparrow,\downarrow]$ and $-\uparrow = \downarrow, -\downarrow=\uparrow$.

**NEED TO ADD 4-FERMION TERMS**

In [ ]:
import numpy as np
from scipy.sparse import dia_matrix
from numpy.linalg import multi_dot
from odeintw import odeintw
from openfermion.ops import FermionOperator, down_index, up_index
from openfermion.hamiltonians import fermi_hubbard
from openfermion.transforms import get_sparse_operator

from openfermion.utils import jw_configuration_state

class Periodic_Lattice(np.ndarray):
    """Creates an n-dimensional ring that joins on boundaries w/ numpy
    
    Required Inputs
        array :: np.array :: n-dim numpy array to use wrap with
    
    Only currently supports single point selections wrapped around the boundary
    """
    def __new__(cls, input_array, lattice_spacing=None):
        """__new__ is called by numpy when and explicit constructor is used:
        obj = MySubClass(params) otherwise we must rely on __array_finalize
         """
        # Input array is an already formed ndarray instance
        # We first cast to be our class type
        obj = np.asarray(input_array).view(cls)
        
        # add the new attribute to the created instance
        obj.lattice_shape = input_array.shape
        obj.lattice_dim = len(input_array.shape)
        obj.lattice_spacing = lattice_spacing
        
        # Finally, we must return the newly created object:
        return obj
    
    def __getitem__(self, index):
        index = self.latticeWrapIdx(index)
        return super(Periodic_Lattice, self).__getitem__(index)
    
    def __setitem__(self, index, item):
        index = self.latticeWrapIdx(index)
        return super(Periodic_Lattice, self).__setitem__(index, item)
    
    def __array_finalize__(self, obj):
        """ ndarray.__new__ passes __array_finalize__ the new object, 
        of our own class (self) as well as the object from which the view has been taken (obj). 
        See
        http://docs.scipy.org/doc/numpy/user/basics.subclassing.html#simple-example-adding-an-extra-attribute-to-ndarray
        for more info
        """
        # ``self`` is a new object resulting from
        # ndarray.__new__(Periodic_Lattice, ...), therefore it only has
        # attributes that the ndarray.__new__ constructor gave it -
        # i.e. those of a standard ndarray.
        #
        # We could have got to the ndarray.__new__ call in 3 ways:
        # From an explicit constructor - e.g. Periodic_Lattice():
        #   1. obj is None
        #       (we're in the middle of the Periodic_Lattice.__new__
        #       constructor, and self.info will be set when we return to
        #       Periodic_Lattice.__new__)
        if obj is None: return
        #   2. From view casting - e.g arr.view(Periodic_Lattice):
        #       obj is arr
        #       (type(obj) can be Periodic_Lattice)
        #   3. From new-from-template - e.g lattice[:3]
        #       type(obj) is Periodic_Lattice
        # 
        # Note that it is here, rather than in the __new__ method,
        # that we set the default value for 'spacing', because this
        # method sees all creation of default objects - with the
        # Periodic_Lattice.__new__ constructor, but also with
        # arr.view(Periodic_Lattice).
        #
        # These are in effect the default values from these operations
        self.lattice_shape = getattr(obj, 'lattice_shape', obj.shape)
        self.lattice_dim = getattr(obj, 'lattice_dim', len(obj.shape))
        self.lattice_spacing = getattr(obj, 'lattice_spacing', None)
        pass
    
    def latticeWrapIdx(self, index):
        """returns periodic lattice index 
        for a given iterable index
        
        Required Inputs:
            index :: iterable :: one integer for each axis
        
        This is NOT compatible with slicing
        """
        if not hasattr(index, '__iter__'): return index         # handle integer slices
        if len(index) != len(self.lattice_shape): return index  # must reference a scalar
        if any(type(i) == slice for i in index): return index   # slices not supported
        if len(index) == len(self.lattice_shape):               # periodic indexing of scalars
            mod_index = tuple(( (i%s + s)%s for i,s in zip(index, self.lattice_shape)))
            return mod_index
        raise ValueError('Unexpected index: {}'.format(index))

def get_jmat_obc(args):
    """
    This is the Jmn hopping matrix with power law decay for open boundary
    conditions.
    
    Parameters:
    args (dict): Dictionary conntaining parameters
                 1. "lattice_size" : Size of lattice
                 2. "beta"         : Long-range power law
                 
    Returns:
    numpy array : Long-Range Hopping matrix with antiperiodic boundary conditions
    """
    N = args["lattice_size"]
    J = dia_matrix((N, N))
    mid_diag = np.floor(N/2).astype(int)
    for i in np.arange(1,mid_diag+1):
        elem = pow(i, -args["beta"])
        J.setdiag(elem, k=i)
        J.setdiag(elem, k=-i)
    for i in np.arange(mid_diag+1, N):
        elem = pow(N-i, -args["beta"])
        J.setdiag(elem, k=i)
        J.setdiag(elem, k=-i)
    return J.toarray()

def get_rhomats_spinfull(psi, lsize):
    N = lsize
    rho_uu, rho_ud, rho_du, rho_dd = \
                psi[0:N**2].reshape((N,N)), psi[N**2:2*N**2].reshape((N,N)), psi[2*N**2:3*N**2].reshape((N,N)), psi[3*N**2:4*N**2].reshape((N,N))
    return rho_uu, rho_ud, rho_du, rho_dd

def get_taumats_spinfull(psi, lsize):
    N = lsize
    tau_uu, tau_ud, tau_du, tau_dd = \
                psi[4*N**2:5*N**2].reshape((N,N)), psi[5*N**2:6*N**2].reshape((N,N)), psi[6*N**2:7*N**2].reshape((N,N)), psi[7*N**2:8*N**2].reshape((N,N))
    return tau_uu, tau_ud, tau_du, tau_dd

def diffmat(A,B):
    return A[:,np.newaxis] - B

# Define the right-hand-side of the differential equation.
def dtwa_hubbard(psi, t, J, U, args):
    lsize = args["lattice_size"]
    ruu, rud, rdu, rdd = get_rhomats_spinfull(psi, lsize)
    tuu, tud, tdu, tdd = get_taumats_spinfull(psi, lsize)
        
    dr_uu = - (1j) * (np.dot(J, ruu) - np.dot(ruu, J)) + (1j) * U * diffmat(np.diag(rdd), np.diag(rdd))
    dr_ud = - (1j) * (np.dot(J, rud) - np.dot(rud, J)) + (1j) * U * diffmat(np.diag(rdd), np.diag(ruu))
    dr_du = - (1j) * (np.dot(J, rdu) - np.dot(rdu, J)) + (1j) * U * diffmat(np.diag(ruu), np.diag(rdd))
    dr_dd = - (1j) * (np.dot(J, rdd) - np.dot(rdd, J)) + (1j) * U * diffmat(np.diag(ruu), np.diag(ruu))
    
    raa_uu = np.tile(np.diag(ruu),(lsize,1))
    raa_dd = np.tile(np.diag(rdd),(lsize,1))
    
    rbb_uu = np.transpose(raa_uu)
    rbb_dd = np.transpose(raa_dd)
    dt_uu = (1j) * (np.dot(J, tuu) - np.dot(tuu.T, J)) - (1j) * U * (tuu * raa_dd - tuu.T * rbb_dd) - (0.5j)* U * (tuu-tuu.T) 
    dt_ud = (1j) * (np.dot(J, tud) - np.dot(tud.T, J)) - (1j) * U * (tud * raa_dd - tud.T * rbb_uu) - (0.5j)* U * (tud-tud.T) 
    dt_du = (1j) * (np.dot(J, tdu) - np.dot(tdu.T, J)) - (1j) * U * (tdu * raa_uu - tdu.T * rbb_dd) - (0.5j)* U * (tdu-tdu.T) 
    dt_dd = (1j) * (np.dot(J, tdd) - np.dot(tdd.T, J)) - (1j) * U * (tdd * raa_uu - tdd.T * rbb_uu) - (0.5j)* U * (tdd-tdd.T) 
    
    return np.concatenate((dr_uu.flatten(), dr_ud.flatten(), dr_du.flatten(), dr_dd.flatten(),\
                                   dt_uu.flatten(), dt_ud.flatten(), dt_du.flatten(), dt_dd.flatten()))

def evolve_dtwa_hubbard(seed, times, site, niter_loc, args):
    (J, U, params) = args
    lsize = params["lattice_size"]
    occ = params["occ_spins"]
    N = lsize
    np.random.seed(seed)
    #Initial conditions
    nsq = lsize * lsize
    psi0 = np.zeros(8 * nsq, dtype=np.complex128)
    rinit_uu = np.zeros((lsize, lsize))
    rinit_dd = np.zeros((lsize, lsize))

    rinit_uu[np.arange(lsize), np.arange(lsize)] = -0.5
    rinit_dd[np.arange(lsize), np.arange(lsize)] = -0.5
    
    nsite = np.zeros_like(times)
    for _ in np.arange(niter_loc):
        #occupied sites are here
        rinit_uu[occ,occ] = np.random.normal(0.5, 0.5, len(occ))
        rinit_dd[occ,occ] = np.random.normal(0.5, 0.5, len(occ))
        psi0[0:N**2] = rinit_uu.flatten()
        psi0[3*N**2:4*N**2] = rinit_dd.flatten()
        psi_t = odeintw(dtwa_hubbard, psi0, times, args=args, Dfun=None)
        #Change this with the spinful observable
        nsite = nsite + np.array([1 + get_rhomats_spinfull(psi, N)[0][site,site] + get_rhomats_spinfull(psi, N)[-1][site,site] for psi in psi_t])/2
    return nsite/niter_loc

def evolve_hubbard_exact(times, site, xy ,args):
    (J, U, params) = args
    lsize = params["lattice_size"]    
    occupied_spins = params["occ_spins"]
    #initially occupied states
    up_state_occ = jw_configuration_state(up_index(occupied_spins), 2 * nsites).flatten()
    down_state_occ = jw_configuration_state(down_index(occupied_spins), 2 * nsites).flatten()
    
    # Number operator at the given site
    upspin = jw_configuration_state(up_index(np.array([0])), 2).flatten() # |\uparrow\rangle
    downspin = jw_configuration_state(down_index(np.array([0])), 2).flatten() #|\down\rangle
    # |\uparrow\rangle \langle \uparrow | + |\downarrow\rangle \langle \downarrow |
    number_op = np.kron(upspin.conjugate().reshape(-1,1), upspin) +\
                np.kron(downspin.conjugate().reshape(-1,1), downspin)   
    num_init = np.kron(np.kron(np.eye(4**site), number_op), np.eye(4**(lsize-site-1)))
    # Long-range Kinetic energy terms
    ke = FermionOperator()
    for i in range(lsize):
        for j in range(i+1, lsize):
            tij = J[i,j]
            iup, jup = up_index(i), up_index(j)
            idn, jdn = down_index(i), down_index(j)
            ke += FermionOperator(((iup, 1), (jup, 0)), tij) + FermionOperator(((jup, 1), (iup, 0)), tij.conjugate())
            ke += FermionOperator(((idn, 1), (jdn, 0)), tij) + FermionOperator(((jdn, 1), (idn, 0)), tij.conjugate())

    # Only interaction. Set tunelling to 0 as tunelling will be long range
    # U -> U/2.0 seems to be necessary, probably to overcount
    interaction = fermi_hubbard(lsize, 1, 0.0, U, particle_hole_symmetry=True)

    # Get scipy.sparse.csc representation and densify it to get the full Hamiltonian
    H = get_sparse_operator(ke).todense() + get_sparse_operator(interaction).todense()
    H = np.asarray(H) # convert from numpy matrix to ndarray

    psi_init = (up_state_occ + down_state_occ + (1j) * np.zeros(4**lsize))/np.sqrt(2) # This is stupid but apparently necessary
    psi_t = odeintw(lambda psi, t, H:np.dot((-1j) * H, psi), psi_init, t, args=(H,), Dfun=None)

    #Just calculate expetation values now
    n_init_av = np.einsum("ij,jk,ik->i", psi_t.conjugate(), num_init, psi_t)
    return n_init_av    

#Plotting stuff
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rc
#Use latex labels
rc('font',**{'family':'sans-serif','sans-serif':['Linux Libertine']})
rc('text', usetex=True)
#Size of fonts
fs = 25
rc({'axes.titlesize': fs})
rc({'axes.labelsize':fs})
rc({'figure.titlesize':fs})
rc({'legend.fontsize': 0.75 * fs})
#Size of figure
rc({'figure.figsize':(15,8)})
rc({'figure.autolayout':True})

print("TWA Dynamics defined and set")

In [ ]:
from multiprocessing import Pool
# Set parameters.
nsites = 4
coulomb = 1.0
beta = 5.0 # Long range exponent
occupied_spins = np.array([0, 1])
site = 1
t = np.linspace(0.0, 6.0, 1000)

params = {"lattice_size": nsites, "beta": beta, "occ_spins":occupied_spins}
niter = 100
nprocs = 12

if __name__ == '__main__':
    p = Pool(processes = nprocs)
    niter_loc = int(niter/nprocs)
    out_all_dtwa = p.starmap(evolve_dtwa_hubbard, [(s, t, site, niter_loc, (get_jmat_obc(params), coulomb, params))\
                                                                                                                   for s in np.arange(nprocs)])
    n_init_av_dtwa = np.average(out_all_dtwa, axis=0)
       
    n_init_av_exact = evolve_hubbard_exact(t, site, None,(get_jmat_obc(params), coulomb, params))
    #Plot the expectation values
    plt.figure(figsize=(15,8))
    plt.title(r"$L = %d$, fill$ = %.2lf$," % (nsites, len(occupied_spins)/nsites) + r" $U/t = %.2lf$" % (coulomb))
    plt.xlabel(r'time $(\hbar/J)$')
    plt.ylabel(r'$\langle n_%d \rangle$' % site)

    plt.plot(t, n_init_av_exact.real,label = "Exact")
    plt.plot(t, n_init_av_dtwa,label = "DTWA")
    plt.legend()